In [ ]:
import pandas as pd
import requests

In [ ]:
#github access token

headers = {
    'Authorization':'*******'
}

In [ ]:
# github rate limit tracking

import datetime

response = requests.get('https://api.github.com/rate_limit', headers=headers)
rate_limit_data = response.json()

rate_limit=rate_limit_data['resources']['core']['limit']
rate_remaining=rate_limit_data['resources']['core']['remaining']
reset_timestamp = rate_limit_data['resources']['core']['reset']
reset_datetime = datetime.datetime.fromtimestamp(reset_timestamp)

print(rate_limit,rate_remaining, reset_datetime)

60 0 2024-10-30 13:22:04


In [ ]:
# step 1 : extract the list of users in Basel with more than 10 followers

city='Basel'
m_follow=10
per_page=100

query=f'location:{city} followers:>{m_follow}'
url=f'https://api.github.com/search/users?q={query}&per_page={per_page}'

users_list=[]

while url:

  response=requests.get(url,headers=headers)
  if response.status_code==200:

    users_data=response.json().get('items',[])
    users_list.extend([item['login'] for item in users_data])

    link_header=response.headers.get('link')

    if link_header and 'rel="next"' in link_header:
      for link in link_header.split(','):
        if 'rel="next"' in link:
          url=link.split(';')[0].strip('<> ')
          break
        else:
          url= None
    else:
      url=None



In [ ]:
print(len(users_list))

349


In [ ]:
users_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TDS/Project 1 - Basel10/users.csv')
users_df.shape[0]

349

In [ ]:
#2 extract the requisite columns for each of the filtered users

req_fields = req_fields=['login','name','company','location','email','hireable','bio','public_repos','followers','following','created_at']

user_fields={}
i=users_df.shape[0]

for user in users_list[i:]:
  url=f'https://api.github.com/users/{user}'
  response=requests.get(url,headers=headers)
  if response.status_code==200:
    user_data=response.json()
    for field in req_fields:
      user_fields[field]=user_data[field]
    users_df.loc[i]=user_fields
    i+=1


In [ ]:
users_df.shape

(349, 11)

In [ ]:
users_df.tail()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
344,alvarogonjim,Alvaro Gonzalez-Jimenez,University of Basel,Basel,NaN,NaN,NaN,23,11,11,2017-02-09T17:00:18Z
345,palsch,Paul Schell,PRODYNA (Schweiz) AG,Basel,NaN,NaN,I am a senior software architect and consultan...,18,11,14,2012-03-21T17:09:18Z
346,dsolsona,Daniel Solsona,Beekeeper,Basel,NaN,NaN,NaN,35,11,3,2011-06-26T16:07:22Z
347,frieder,Frieder Heugel,NaN,"Basel, Switzerland",NaN,NaN,NaN,10,11,0,2011-01-31T10:19:09Z
348,dpdawson,Daniel Dawson,Google,"Basel, Switzerland",NaN,True,NaN,27,11,88,2010-11-16T21:28:17Z


In [ ]:
users_df.to_csv('/content/drive/MyDrive/Colab Notebooks/TDS/Project 1 - Basel10/users.csv',index=False)